# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\tag{2}
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otoczenia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\tag{3}
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3. Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
from scipy import signal
import cv2
import copy

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

#TODO Samodzielna

mat = loadmat('MR_data.mat')

I = mat['I_noisy1']
Input_2 = mat['I_noisy2']
Input_3 = mat['I_noisy3']
Input_4 = mat['I_noisy4']
Input_0 = mat['I_noisefree']

plt.imshow(I, 'gray')
plt.axis('off')
plt.show()

# To obrazowanie to rezonans magnetyczny.

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
#TODO Samodzielna
def fgaussian(size, sigma):
    m = n = size
    h, k = m // 2, n // 2
    x, y = np.mgrid[-h:h+1, -k:k+1]
    g = np.exp(-(x**2 + y**2)/(2*sigma**2))
    return g /g.sum() 
    
def mesh(size, sigma):
    m = n = size
    h, k = m//2, n//2
    x, y = np.mgrid[-h:h+1, -k:k+1]
    g = np.exp(-(x**2 + y**2)/(2*sigma**2))
    return g 

In [ ]:
size = 5
n=3
sigma_s = 0.6

M = fgaussian(n, sigma_s)
M = M/sum(sum(M));       
I_Conv = I.copy()

def wsp(okno, filtr, wariancja):
    A,B = okno.shape
    piksel = 0
    x = [A // 2, B // 2]
    for i in range(A):
        for j in range(B):
            AB = [i, j]
            y = np.sqrt(((x[0] - i)**2) + ((x[1] - j)**2))
            g = np.exp(-(y**2)/(2*(wariancja**2)))
            piksel = piksel + g * okno[i, j]
    piksel = piksel / filtr.sum()
    return piksel


def konwolucja(obraz, okno, wariancja):
    filtr = mesh(5, wariancja)
    IConv = obraz.copy()
    (X,Y) = IConv.shape
    polowa_okna = okno // 2
    for i in range(0 + polowa_okna, X - polowa_okna):
        for j in range(0 + polowa_okna, Y - polowa_okna):
            okno = IConv[i - polowa_okna : i + polowa_okna + 1, j - polowa_okna : j + polowa_okna + 1]
            new_pixel = wsp(okno, filtr, wariancja)
            IConv[i, j] = new_pixel
    return IConv

img_1 = konwolucja(I, size, sigma_s)

plt.figure(figsize=(15,5))
plt.subplot(131),
plt.imshow(I,"gray"),
plt.title('Obraz oryginalny',fontsize=20)
plt.xticks([]), plt.yticks([])

plt.subplot(132),
plt.imshow(img_1,"gray"),
plt.title('Obraz przefiltrowany',fontsize=20)
plt.xticks([]), plt.yticks([])
#Obraz wynikowy stał się jaśniejsy od obrazu oryginalnego.

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gaussowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def exp(y, sigma):
  return np.exp(-(y*y/2*sigma*sigma))

def gamma(tab, sigma, Ix):
  tab_new = np.zeros((3, 3))
  for i in range(len(tab)):
    for j in range(len(tab[0])):
      tab_new[i, j] = exp(abs(tab[i, j] - Ix), sigma)
  return tab_new

def wsp_2(okno, filtr, wariancja, delta_r):
    A,B = okno.shape
    piksel_2 = 0
    normalizacja = 0
    
    tab = [A // 2, B // 2]
    for i in range(A):
        for j in range(B):
            AB = [i, j]
            
            y = np.sqrt(((tab[0] - i)**2) + ((tab[1] - j)**2))
            gauss = np.exp(-(y**2) / (2*(wariancja**2)))
            
            roznica = np.abs(okno[A // 2, B // 2] - okno[i, j])
            gauss_roznica = np.exp(-(roznica**2) / (2*(delta_r**2)))
            
            piksel_2 = piksel_2 + gauss * gauss_roznica * okno[i, j]
            normalizacja = normalizacja+(gauss * gauss_roznica)
    piksel_2 = piksel_2 / normalizacja
    return piksel_2
        


In [ ]:
def bilateralna(obraz, okno, wariancja, delta_r):
    filtr = mesh(okno, wariancja)
    IConvol = obraz.copy()
    (X,Y) = IConvol.shape
    polowa_okna = okno//2
    for i in range(0 + polowa_okna, X - polowa_okna):
        for j in range(0 + polowa_okna, Y - polowa_okna):
            okno = IConvol[i - polowa_okna : i + polowa_okna + 1, j - polowa_okna : j + polowa_okna + 1]
            nowy_pixel = wsp_2(okno, filtr, wariancja, delta_r)
            IConvol[i,j] = nowy_pixel
    return IConvol

In [ ]:

konwolucja_1 = konwolucja(I, 5, 0.7)

bilateralna = bilateralna(I, 5, 0.7, 20)

I_1_gaus = signal.convolve2d(I, fgaussian(5, 0.7), mode='same')

f, ax = plt.subplots(2, 2, figsize=(18,16))
ax[0,0].imshow(I, 'gray')
ax[0,0].set_title("Oryginał",fontsize=20)
ax[0,0].axis('off')

ax[1,0].imshow(konwolucja_1, 'gray')
ax[1,0].set_title("Konwolucja",fontsize=20)
ax[1,0].axis('off')

ax[0,1].imshow(bilateralna, 'gray')
ax[0,1].set_title("Filtracja bilateralna",fontsize=20)
ax[0,1].axis('off')

ax[1,1].imshow(I_1_gaus, 'gray')
ax[1,1].set_title("Konwolucja z biblioteki cv2",fontsize=20)
ax[1,1].axis('off')